In [1]:
import os
import gc
import math

# numpy
import numpy as np
import numpy.ma as ma

from jax import vmap, grad, jit, random
from jax.config import config
# from jax.experimental import optimizers
import jax.numpy as jnp
from jax.tree_util import tree_map, tree_flatten, tree_unflatten, tree_leaves

config.update("jax_enable_x64", True)

# from skimage.restoration import estimate_sigma
from skimage.transform import resize

### CRPS
import properscoring as ps

### SSIM and PSNR
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import filters

# plotting
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')

from matplotlib.gridspec import GridSpec
from matplotlib import colors
from cartopy import config
import cartopy.crs as ccrs

# netCDF
import netCDF4 as nc

from tqdm.notebook import tqdm
from tqdm.notebook import trange

import pickle

In [7]:
sgpr = [np.load(f'tas_predictions/sgpr_tas_{m}.npz.npy') for m in trange(nmod)]
sgpr_var = [np.load(f'tas_predictions/sgpr_tas_{m}_var.npz.npy') for m in trange(nmod)]

xreg = [np.load(f'tas_predictions/xreg_tas_{m}.npz.npy') for m in trange(nmod)]
xreg_var = [np.load(f'tas_predictions/xreg_tas_{m}_var.npz.npy') for m in trange(nmod)]

xbar = [np.load(f'tas_predictions/xbar_tas_{m}.npz.npy') for m in trange(nmod)]
xbar_var = [np.load(f'tas_predictions/xbar_tas_{m}_var.npz.npy') for m in trange(nmod)]

xwea = [np.load(f'tas_predictions/xwea_tas_{m}.npz.npy') for m in trange(nmod)]
xwea_var = [np.load(f'tas_predictions/xbar_tas_{m}_var.npz.npy') for m in trange(nmod)]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [25]:
sgpr_crps = np.zeros((decades, nmod))
xreg_crps = np.zeros((decades, nmod))
xbar_crps = np.zeros((decades, nmod))

sgpr_cover = np.zeros((decades, nmod))
xreg_cover = np.zeros((decades, nmod))
xbar_cover = np.zeros((decades, nmod))

for m1 in trange(8, 9):
    ytest = np.array(xrcp[m1])[nval:ntest]
    
    nlat = ytest.shape[1]
    scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))
    
    for d in range(decades):
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')
        
        ### crps
        sgpr_crps_dec = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
        xreg_crps_dec = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
        xbar_crps_dec = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)
    

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
m1 = 0

ytest = np.array(xrcp[m1])[nval:ntest]
nlat = ytest.shape[1]
scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))

d = 0
ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')

ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')

### crps
sgpr_crps0 = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
xreg_crps0 = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
xbar_crps0 = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)

d = 7
ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')

ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')

### crps
sgpr_crps1 = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
xreg_crps1 = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
xbar_crps1 = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)

In [9]:
crps_diff = []

cr1 = np.mean(sgpr_crps0 * scale[None,:,None], axis = 0)
cr2 = np.mean(xreg_crps0 * scale[None,:,None], axis = 0)
crps_diff.append(cr2 - cr1)


cr1 = np.mean(sgpr_crps1 * scale[None,:,None], axis = 0)
cr2 = np.mean(xreg_crps1 * scale[None,:,None], axis = 0)
crps_diff.append(cr2 - cr1)

In [ ]:
m1 = 0

ytest = np.array(xrcp[m1])[nval:ntest]
nlat = ytest.shape[1]
scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))

ytest_dec = ytest.astype('float32')
sgpr_dec = sgpr[m1].astype('float32')
xwea_dec = xwea[m1].astype('float32')
xreg_dec = xreg[m1].astype('float32')

ytest_dec = ytest.astype('float32')
sgpr_var_dec = sgpr_var[m1].astype('float32')
xwea_var_dec = xwea_var[m1].astype('float32')
xreg_var_dec = xreg_var[m1].astype('float32')

### crps
sgpr_crps0 = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
xreg_crps0 = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
# xwea_crps0 = ps.crps_gaussian(ytest_dec, xwea_dec, xwea_var_dec)

crps_diff = []

cr1 = np.mean(sgpr_crps0 * scale[None,:,None], axis = 0)
cr2 = np.mean(xreg_crps0 * scale[None,:,None], axis = 0)
# cr2 = np.mean(xwea_crps0 * scale[None,:,None], axis = 0)

crps_diff.append((cr2 - cr1))
crps_diff.append((cr1 - cr2) < 0)


ystd = np.std(ytest_dec, axis = 0)
ystd = (ystd - np.mean(ystd))
crps_diff.append((ystd > 0))

##########


fig = plt.figure(constrained_layout = True, figsize = (16, 4))
gs = GridSpec(1, 5, width_ratios = [1, 0.05, 0.1, 1, 1], figure = fig)
data_crs = ccrs.PlateCarree(central_longitude=180)
plot_crs = ccrs.Robinson(central_longitude=180)

titles = ['CRPS improvement (T2M)', 'NN-GPR improves CRPS (T2M)', 'High variance (T2M)']
titles = ['LM - NN-GPR\nCRPS differences (T2M)', 'NN-GPR has lower\nCRPS (T2M)', 'Above average\nvariance (T2M)']

divnorm=colors.SymLogNorm(0.1, vmin=-1.5, vmax=1.5)
ax = plt.subplot(gs[0], projection=plot_crs)
ax.set_title(titles[0], fontsize = 19)


ax.text(0.035, 0.932, 'a', fontsize=25, fontweight='bold', transform=plt.gcf().transFigure)

im = ax.imshow(crps_diff[0], 
      cmap = newcmp, 
      origin='upper', 
      transform = data_crs,
      norm = divnorm)
ax.coastlines()
ax.set_aspect('auto', adjustable=None)

ax = plt.subplot(gs[1])
cbar = plt.colorbar(im, cax=ax)
cbar.ax.tick_params(labelsize=12)


ax.text(0.42, 0.932, 'b', fontsize=25, fontweight='bold', transform=plt.gcf().transFigure)


cmap = colors.ListedColormap(['white', 'maroon'])
bounds=[0,0.5,1]
divnorm = colors.BoundaryNorm(bounds, cmap.N)

for k in range(2):
    ax = plt.subplot(gs[k+3], projection=plot_crs)
    ax.set_title(titles[k+1], fontsize = 19)
    im = ax.imshow(crps_diff[k+1], 
          cmap = cmap, 
          origin='upper', 
          transform = data_crs,
          extent = [-180, 180, -90, 90],
          norm = divnorm)
    ax.coastlines()
    ax.set_aspect('auto', adjustable=None)
    
# ax.scatter(0,0, c = 'maroon', label = 'True', marker='s')
# ax.scatter(None,None, c = 'maroon', label = 'True', marker='s')
# ax.legend(loc='lower right', bbox_to_anchor=[1, 0, 0.3, 0], facecolor = 'white', fontsize = 14)
plt.show()

In [ ]:
sgpr = [np.load(f'pr_predictions/sgpr_pr_{m}.npz.npy') for m in trange(nmod)]
sgpr_var = [np.load(f'pr_predictions/sgpr_pr_{m}_var.npz.npy') for m in trange(nmod)]

xreg = [np.load(f'pr_predictions/xreg_pr_{m}.npz.npy') for m in trange(nmod)]
xreg_var = [np.load(f'pr_predictions/xreg_pr_{m}_var.npz.npy') for m in trange(nmod)]

xbar = [np.load(f'pr_predictions/xbar_pr_{m}.npz.npy') for m in trange(nmod)]
xbar_var = [np.load(f'pr_predictions/xbar_pr_{m}_var.npz.npy') for m in trange(nmod)]

xwea = [np.load(f'pr_predictions/xwea_pr_{m}.npz.npy') for m in trange(nmod)]
xwea_var = [np.load(f'pr_predictions/xbar_pr_{m}_var.npz.npy') for m in trange(nmod)]

In [ ]:
sgpr_crps = np.zeros((decades, nmod))
xreg_crps = np.zeros((decades, nmod))
xbar_crps = np.zeros((decades, nmod))

sgpr_cover = np.zeros((decades, nmod))
xreg_cover = np.zeros((decades, nmod))
xbar_cover = np.zeros((decades, nmod))

for m1 in trange(8, 9):
    ytest = np.array(xrcp[m1])[nval:ntest]
    
    nlat = ytest.shape[1]
    scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))
    
    for d in range(decades):
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')
        
        ### crps
        sgpr_crps_dec = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
        xreg_crps_dec = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
        xbar_crps_dec = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)
    

In [ ]:
m1 = 0

ytest = np.array(xrcp[m1])[nval:ntest]
nlat = ytest.shape[1]
scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))

d = 0
ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')

ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')

### crps
sgpr_crps0 = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
xreg_crps0 = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
xbar_crps0 = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)

d = 7
ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')

ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')

### crps
sgpr_crps1 = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
xreg_crps1 = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
xbar_crps1 = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)